### 该算法首先将所有点作为一个簇，然后将该簇一分为二，之后选择其中一个簇继续进行划分，选择的标准取决于，该簇一分为二后可以最大程度地降低SSE的值。
---
### 伪代码：
```python
将所有点看成一个簇
当簇的数量小于k时：
    对于每一个簇：
        计算总误差SSE
        在当前簇上进行k均值聚类（K=2）
        计算将该簇一分为二后的总误差
    选择使得误差最小的那个簇进行划分操作
```

#### 程序清单10-3 二分K-均值聚类算法

In [1]:
# 需要k均值聚类的所有函数，这里复制过来：
import numpy as np


def loadDataSet(filename):
    """数据加载函数"""
    dataMat=[]
    fr=open(filename)
    for line in fr.readlines():
        curLine=line.strip().split('\t')
        fline=list(map(float, curLine))
        dataMat.append(fline)
    return dataMat


def distEclud(vecA,vecB):
    """计算两个向量之间的欧几里得距离"""
    return np.sqrt(np.sum((vecA-vecB)**2))


def randCent(dataSet,k):
    """初始化簇质心,k是随机生成的质心的个数"""
    n=dataSet.shape[1]
    cent=np.mat(np.zeros((k,n)))    # 初始化k个质心
    for j in range(n):
        minJ=dataSet[:,j].min()
        maxJ=dataSet[:,j].max()
        rangeJ=float(maxJ-minJ)    # 第j个特征的范围
        cent[:,j]=minJ+rangeJ*np.random.rand(k,1)
    return cent


def kMeans(dataSet, k,distMeas=distEclud,createCent=randCent):
    m=dataSet.shape[0]
    clusterAssment=np.mat(np.zeros((m,2)))    # 第一列：每个样本所属的类标签；第二列：存储误差
    center=createCent(dataSet,k)    # 随机初始化各个簇的质心
    clusterChanged=True
    
    while clusterChanged:
        clusterChanged=False
        for i in range(m):    # 遍历每个样本，寻找其属于哪个中心（哪个簇）
            minDist=np.inf
            minIndex=-1
            for j in range(k):   # 对每个质心，寻找距离该样本最短的质心
                # 这里需要注意，datMat[0,:]
                distJI = distEclud(center[j, :].A[0], dataSet[i, :].A[0])    # 注意，datMat[0,:]是一个二维的矩阵，需转换成向量
                if distJI<minDist:
                    minDist=distJI
                    minIndex=j    # 更新该样本属于哪个中心
            # 现在对于该样本已经找到了其属于哪个簇中心
            if clusterAssment[i,0]!=minIndex:    # 样本所属的簇发生改变：
                clusterChanged=True
            clusterAssment[i,:]=minIndex,minDist**2    # 更新簇结果分配矩阵
        # print(center)
        
        for c in range(k):
            ptsInClust=dataSet[np.nonzero(clusterAssment[:,0].A==c)[0]]    # 获取属于簇c的所有样本点
            center[c,:]=np.mean(ptsInClust,axis=0)    # 更新各个簇的质心
            
    return center,clusterAssment


In [ ]:
def biKmeans(dataSet:np.matrix,k:int,distMeas=distEclud):
    m=dataSet.shape[0]
    clusterAssignment=np.mat(np.zeros((m,2)))
    center0=dataSet.mean(axis=0)    # 初始化聚类中心
    centList=[center0]
    
    for j in range(m):
        clusterAssignment[:,1]=distEclud(center0,dataSet[j,:])**2    # 这里的这个**2会报错吗
        
    while len(centList)<k:
        lowestSSE=np.inf
        for i in range(len(centList)):
            # 获取该簇下的数据
            ptsInCurrCenter=dataSet[np.nonzero(clusterAssignment[:,0]==i)[0],:]
            # 对该簇下的数据进行k=2的kMeans聚类
            centMat2,splitClustAss=kMeans(ptsInCurrCenter,k=2)    # 尝试划分当前簇
            sseSplit=np.sum(splitClustAss[:,1],axis=0)    # 误差计算
            # 计算不属于当前簇的其他样本（其实就是另一个簇）的总误差
            sseNotSplit=np.sum(clusterAssignment[:,1][np.nonzero(clusterAssignment.A[:,0]!=i)[0]])
            
            if sseSplit+sseNotSplit<lowestSSE:
                bestCentToSplit=i    # 更新最佳分裂簇
                bestNewCents=centMat2    # 更新最佳簇中心点
                bestClustAss=splitClustAss.copy()
                lowestSSE=sseSplit+sseNotSplit    # 最小误差更新
                
                # 更新簇的分配结果
                bestClustAss[np.nonzero(bestClustAss.A[:,0]==1)[0],0]=len(centList)
                bestClustAss[np.nonzero(bestClustAss.A[:,0]==0)[0],0]=bestCentToSplit
                centList[bestCentToSplit]=bestNewCents[0,:]
                centList.append(bestNewCents[1,:])
        